# Creating QCoDeS instrument drivers

In [1]:
# most of the drivers only need a couple of these... moved all up here for clarity below
from time import sleep, time
import numpy as np
import ctypes  # only for DLL-based instrument

import qcodes as qc
from qcodes import (Instrument, VisaInstrument,
                    ManualParameter, MultiParameter,
                    validators as vals)

## Base Classes

There are 3 available:
- `VisaInstrument` - for most instruments that communicate over a text channel (ethernet, GPIB, serial, USB...) that do not have a custom DLL or other driver to manage low-level commands.
- `IPInstrument` - a specialized driver just for ethernet connections. Otherwise works nearly identically to `VisaInstrument`.
- `Instrument` - superclass of both `VisaInstrument` and `IPInstrument`, use this if you do not communicate over a text channel, for example:
  - PCI cards with their own DLLs
  - Instruments with only manual controls.

## VisaInstrument: Simple example
The Weinschel 8320 driver is about as basic a driver as you can get. It only defines one parameter, "attenuation". All the comments here are my additions to describe what's happening.

In [2]:
class Weinschel_8320(VisaInstrument):
    """
    QCoDeS driver for the stepped attenuator
    Weinschel is formerly known as Aeroflex/Weinschel
    """

    # all instrument constructors should accept **kwargs and pass them on to
    # super().__init__. This is important for the deprecated multiprocessing
    # architecture, so may change in the near future.
    def __init__(self, name, address, **kwargs):
        # supplying the terminator means you don't need to remove it from every response
        super().__init__(name, address, terminator='\r', **kwargs)

        self.add_parameter('attenuation', units='dB',
                           # the value you set will be inserted in this command with
                           # regular python string substitution. The instrument wants
                           # an integer zero-padded to 2 digits. For robustness, don't
                           # assume you'll get an integer input though - try to allow
                           # floats (as opposed to {:0=2d})
                           set_cmd='ATTN ALL {:02.0f}',
                           get_cmd='ATTN? 1',
                           # setting any attenuation other than 0, 2, ... 60 will error.
                           vals=vals.Enum(*np.arange(0, 60.1, 2).tolist()),
                           # the return value of get() is a string, but we want to
                           # turn it into a (float) number
                           get_parser=float)

        # it's a good idea to call connect_message at the end of your constructor.
        # this calls the 'IDN' parameter that the base Instrument class creates for
        # every instrument (you can override the `get_idn` method if it doesn't work
        # in the standard VISA form for your instrument) which serves two purposes:
        # 1) verifies that you are connected to the instrument
        # 2) gets the ID info so it will be included with metadata snapshots later.
        self.connect_message()

# instantiating and using this instrument (commented out because I can't actually do it!)
#
# from qcodes.instrument_drivers.weinschel.Weinschel_8320 import Weinschel_8320
# weinschel = Weinschel_8320('w8320_1', 'TCPIP0::172.20.2.212::inst0::INSTR')
# weinschel.attenuation(40)

## VisaInstrument: a more involved example
the K2600 breaks one physical instrument into two software instruments, one for each channel. It:
- has a custom IDN parser
- transforms the ask and write commands to simplify them and insert the channel.
- demonstrates `val_mapping` for bidirectional encoding of enumerated parameters

I've removed some parameters for brevity.

Note the nice docstring that describes the status of the code!

In [3]:
class Keithley_2600(VisaInstrument):
    """
    channel: use channel 'a' or 'b'

    This is the qcodes driver for the Keithley_2600 Source-Meter series,
    tested with Keithley_2614B

    Status: beta-version.
        TODO:
        - Add all parameters that are in the manual
        - range and limit should be set according to mode
        - add ramping and such stuff

    """
    def __init__(self, name, address, channel, **kwargs):
        super().__init__(name, address, terminator='\n', **kwargs)
        self._channel = channel

        self.add_parameter('volt', get_cmd='measure.v()',
                           get_parser=float, set_cmd='source.levelv={:.8f}',
                           label='Voltage',
                           units='V')
        self.add_parameter('curr', get_cmd='measure.i()',
                           get_parser=float, set_cmd='source.leveli={:.8f}',
                           label='Current',
                           units='A')
        self.add_parameter('mode',
                           get_cmd='source.func',
                           set_cmd='source.func={:d}',
                           # the K2600 interprets '0' as current and '1' as voltage
                           # but we want to type: k2600a.mode('current'), not .mode(0)
                           # val_mapping describes, all in one dictionary:
                           # - the valid set values ('current' and 'voltage')
                           # - the encoding to instrument codes (current->0, voltage->1)
                           # - the decoding back to parameter values (0->current, 1->voltage)
                           # `ask` returns a string, but because integer encodings are so
                           # common we automatically check if the string is an integer
                           # when you use val_mapping
                           val_mapping={'current': 0, 'voltage': 1})
        self.add_parameter('output',
                           get_cmd='source.output',
                           set_cmd='source.output={:d}',
                           # output on/off has another val_mapping
                           val_mapping={'on':  1, 'off': 0})

        self.connect_message()

    def get_idn(self):
        IDN = self.ask_raw('*IDN?')
        vendor, model, serial, firmware = map(str.strip, IDN.split(','))
        model = model[6:]

        IDN = {'vendor': vendor, 'model': model,
               'serial': serial, 'firmware': firmware}
        return IDN

    # Reset method. Since this is a simple command to write, this could also have been:
    # self.add_function('reset', call_cmd='reset()')
    def reset(self):
        self.write('reset()')

    # transformation of ask and write commands. From the Instrument.ask docstring:
    # Subclasses that transform ``cmd`` should override ``ask``, and in
    # it call ``super().ask(new_cmd)``. Subclasses that define a new
    # hardware communication should instead override ``ask_raw``.
    def ask(self, cmd):
        return super().ask('print(smu{:s}.{:s})'.format(self._channel, cmd))

    def write(self, cmd):
        super().write('smu{:s}.{:s}'.format(self._channel, cmd))

## DLL-based instruments
The Alazar cards use their own DLL. C interfaces tend to need a lot of boilerplate, so I'm not going to include it all. The key is: use `Instrument` directly, load the DLL, and have parameters interact with it.

In [4]:
class AlazarTech_ATS(Instrument):
    dll_path = 'C:\\WINDOWS\\System32\\ATSApi'
    def __init__(self, name, system_id=1, board_id=1, dll_path=None, **kwargs):
        super().__init__(name, **kwargs)
        
        # connect to the DLL
        self._ATS_dll = ctypes.cdll.LoadLibrary(dll_path or self.dll_path)

        self._handle = self._ATS_dll.AlazarGetBoardBySystemID(system_id,
                                                              board_id)
        if not self._handle:
            raise Exception('AlazarTech_ATS not found at '
                            'system {}, board {}'.format(system_id, board_id))

        self.buffer_list = []
        
        # the Alazar driver includes its own parameter class to hold values
        # until later config is called, and warn if you try to read a value
        # that hasn't been sent to config.
        self.add_parameter(name='clock_source',
                           parameter_class=AlazarParameter,
                           label='Clock Source',
                           unit=None,
                           value='INTERNAL_CLOCK',
                           byte_to_value_dict={1: 'INTERNAL_CLOCK',
                                               4: 'SLOW_EXTERNAL_CLOCK',
                                               5: 'EXTERNAL_CLOCK_AC',
                                               7: 'EXTERNAL_CLOCK_10_MHz_REF'})
        
        # etc...

## Manual instruments
A totally manual instrument (like the ithaco 1211) will contain only `ManualParameter`s. Some instruments may have a mix of manual and standard parameters. Here we also define a new `CurrentParameter` class that uses the ithaco parameters to convert a measured voltage to a current.

In [5]:
class CurrentParameter(MultiParameter):
    """
    Current measurement via an Ithaco preamp and a measured voltage.

    To be used when you feed a current into the Ithaco, send the Ithaco's
    output voltage to a lockin or other voltage amplifier, and you have
    the voltage reading from that amplifier as a qcodes parameter.

    ``CurrentParameter.get()`` returns ``(voltage_raw, current)``

    Args:
        measured_param (Parameter): a gettable parameter returning the
            voltage read from the Ithaco output.

        c_amp_ins (Ithaco_1211): an Ithaco instance where you manually
            maintain the present settings of the real Ithaco amp.

        name (str): the name of the current output. Default 'curr'.
            Also used as the name of the whole parameter.
    """
    def __init__(self, measured_param, c_amp_ins, name='curr'):
        p_name = measured_param.name

        p_label = getattr(measured_param, 'label', None)
        p_unit = getattr(measured_param, 'units', None)

        super().__init__(name=name, names=(p_name+'_raw', name),
                         shapes=((), ()),
                         labels=(p_label, 'Current'),
                         units=(p_unit, 'A'))

        self._measured_param = measured_param
        self._instrument = c_amp_ins

    def get(self):
        volt = self._measured_param.get()
        current = (self._instrument.sens.get() *
                   self._instrument.sens_factor.get()) * volt

        if self._instrument.invert.get():
            current *= -1

        value = (volt, current)
        self._save_val(value)
        return value


class Ithaco_1211(Instrument):
    """
    This is the qcodes driver for the Ithaco 1211 Current-preamplifier.

    This is a virtual driver only and will not talk to your instrument.
    """
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)

        # ManualParameter has an "initial_value" kwarg, but if you use this
        # you must be careful to check that it's correct before relying on it.
        # if you don't set initial_value, it will start out as None.
        self.add_parameter('sens',
                           parameter_class=ManualParameter,
                           initial_value=1e-8,
                           label='Sensitivity',
                           units='A/V',
                           vals=vals.Enum(1e-11, 1e-10, 1e-09, 1e-08, 1e-07,
                                          1e-06, 1e-05, 1e-4, 1e-3))

        self.add_parameter('invert',
                           parameter_class=ManualParameter,
                           initial_value=True,
                           label='Inverted output',
                           vals=vals.Bool())

        self.add_parameter('sens_factor',
                           parameter_class=ManualParameter,
                           initial_value=1,
                           label='Sensitivity factor',
                           units=None,
                           vals=vals.Enum(0.1, 1, 10))

        self.add_parameter('suppression',
                           parameter_class=ManualParameter,
                           initial_value=1e-7,
                           label='Suppression',
                           units='A',
                           vals=vals.Enum(1e-10, 1e-09, 1e-08, 1e-07, 1e-06,
                                          1e-05, 1e-4, 1e-3))

        self.add_parameter('risetime',
                           parameter_class=ManualParameter,
                           initial_value=0.3,
                           label='Rise Time',
                           units='msec',
                           vals=vals.Enum(0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30,
                                          100, 300, 1000))

    def get_idn(self):
        return {'vendor': 'Ithaco (DL Instruments)', 'model': '1211',
                'serial': None, 'firmware': None}

## Custom Parameter classes

When you call:
```
self.add_parameter(name, **kwargs)
```
you create a `StandardParameter`. But with the `parameter_class` kwarg you can invoke any class you want:
```
self.add_parameter(name, parameter_class=OtherClass, **kwargs)
```

- `StandardParameter` handles most common instrument settings and measurements.
  - Accepts get and/or set commands as either strings for the instrument's `ask` and `write` methods, or functions/methods.
  - Has options for translating between instrument codes and more meaningful data values
  - Supports software-controlled ramping
- `ManualParameter` is for things you don't control electronically, like physical knobs. When you *manually* change the knob, you must *manually* change it in software too.
- Any other parameter class may be used in `add_parameter`, if it accepts `name` and `instrument` as constructor kwargs. Generally these should subclass `Parameter`, `ArrayParameter`, or `MultiParameter`.

## Dynamically adding and removing parameters

Sometimes when conditions change (for example, the mode of operation of the instrument is changed from current to voltage measurement) you want different parameters to be available.

To delete existing parameters:
```
del self.parameters[name_to_delete]
```
And to add more, do the same thing as you did initially:
```
self.add_parameter(new_name, **kwargs)
```

## Functions

a `Function` is some action that the instrument can do, but doesn't correspond to a particular state variable or acquired data. For example, reset, trigger, or beep. Complex cases are best handled by adding your own methods to the instrument, but simple cases (particularly simple commands like `*RST`) can use `add_function`, which acts like a simplified `add_parameter`.

## Organization

Your drivers do not need to be part of QCoDeS in order to use them with QCoDeS, but we strongly encourage you to contribute them to the project. That way we prevent duplication of effort, and you will likely get help making the driver better, with more features and better code.

Make one driver per module, inside a directory named for the company (or institution), within the `instrument_drivers` directory, following the convention:

`instrument_drivers.<company>.<model>.<company>_<model>`
- example: `instrument_drivers.AlazarTech.ATS9870.AlazarTech_ATS9870`

Although the class name can be just the model if it is globally unambiguous. For example:
- example: `instrument_drivers.stanford_research.SR560.SR560`

And note that due to mergers, some drivers may not be in the folder you expect:
- example: `instrument_drivers.tektronix.Keithley_2600.Keithley_2600`